<h1>Table of Contents<span class="tocSkip"></span></h1>


# Introduction
<hr style="border:2px solid black"> </hr>


**What?** Pydantic introduction



# Imports
<hr style="border:2px solid black"> </hr>

In [37]:
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel
from pydantic import ValidationError
from pydantic import Field

# With pydantic
<hr style="border:2px solid black"> </hr>

In [21]:
class User(BaseModel):
    item_id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

In [22]:
external_data = {
    'item_id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}

In [23]:
user = User(**external_data)

In [25]:
user.dict()

{'item_id': 123,
 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'friends': [1, 2, 3],
 'name': 'John Doe'}

# How pydantic fails
<hr style="border:2px solid black"> </hr>

In [26]:
external_data = {
    'item_id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, 'not number'],
}

In [27]:
user = User(**external_data)

ValidationError: 1 validation error for User
friends -> 2
  value is not a valid integer (type=type_error.integer)

In [28]:
try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())

[
  {
    "loc": [
      "item_id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


# `Field` and some weird behaviour with "_"
<hr style="border:2px solid black"> </hr>

    
- pay attention if your variable start with an `_` as in `_id`
- Surprisingly (or at least surprising to me), Pydantic hides fields that start with an underscore (regardless of how you try to access them).
- To be able to use those variables you need to use an alias with `Field`



In [29]:
class User(BaseModel):
    _id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []

In [32]:
external_data = {
    '_id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, 3],
}

In [33]:
user = User(**external_data)

In [35]:
# where is _id?
user.dict()

{'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
 'friends': [1, 2, 3],
 'name': 'John Doe'}

In [102]:
class Item(BaseModel):
    _id: str
    is_available: bool  # 1

In [103]:
item1 = Item(_id='test-item-id', is_available=True) 

In [104]:
# where is _id?
item1.dict()

{'is_available': True}

In [108]:
class Item(BaseModel):
    item_id: str = Field(alias="_id")
    is_available: bool  # 1

In [109]:
item1 = Item(_id='test-item-id', is_available=True) 

In [110]:
# where is _id?
item1.dict()

{'item_id': 'test-item-id', 'is_available': True}

# References
<hr style="border:2px solid black"> </hr>


- [Pydantic official documentation](https://docs.pydantic.dev/latest/)
- [Cool things you can do with Pydantic](https://medium.com/swlh/cool-things-you-can-do-with-pydantic-fc1c948fbde0)

